In [ ]:
import os

# Byt till din PythonML-mapp
os.chdir("/Users/gabrieledholm/Documents/PythonML")
print("Nuvarande arbetsmapp:", os.getcwd())

# Kontrollera att din dataset-mapp finns
dataset_path = "NewLogoPictures"
print("Finns NewLogoPictures?:", os.path.exists(dataset_path))
print("Innehåll:", os.listdir(dataset_path))

In [ ]:
images_path = os.path.join(dataset_path, "images")
labels_path = os.path.join(dataset_path, "labels")

print("Antal bilder:", len(os.listdir(images_path)) if os.path.exists(images_path) else 0)
print("Antal label-filer:", len(os.listdir(labels_path)) if os.path.exists(labels_path) else 0)


In [ ]:
data_yaml = f"""
train: {dataset_path}/images
val: {dataset_path}/images
nc: 1
names: ['hp_logo']
"""


yaml_path = "data.yaml"   
with open(yaml_path, "w") as f:
    f.write(data_yaml)

print("data.yaml skapad på:", yaml_path)


In [ ]:
from ultralytics import YOLO

# Ladda en förtränad YOLO-modell
model = YOLO("yolov8n.pt")

# Träna modellen med din dataset
model.train(
    data="/Users/gabrieledholm/Documents/PythonML/data.yaml",  # rätt plats!
    epochs=50,
    imgsz=640,
    batch=16,
    name="newHP_Logo_train",
    exist_ok=True
)

In [ ]:
import os
import cv2
from ultralytics import YOLO

# ----------------------
# Inställningar
# ----------------------
video_input = "/Users/gabrieledholm/Documents/PythonML/Pitstop.mp4"  # Originalvideo
video_output = "/Users/gabrieledholm/Documents/PythonML/Pitstop_result_final1.mp4"  # Sparas här

# Säkerställ att output-mappen finns
output_dir = os.path.dirname(video_output)
os.makedirs(output_dir, exist_ok=True)

# Ladda den tränade YOLO-modellen
model = YOLO("runs/detect/newHP_Logo_train/weights/best.pt")

# Öppna videon
cap = cv2.VideoCapture(video_input)
if not cap.isOpened():
    raise RuntimeError(f"Kunde inte öppna videon: {video_input}")

fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:
    fps = 30  # default om FPS inte kan hämtas
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Skapa VideoWriter med H.264 codec
fourcc = cv2.VideoWriter_fourcc(*'avc1')
out = cv2.VideoWriter(video_output, fourcc, fps, (width, height))
if not out.isOpened():
    raise RuntimeError(f"Kunde inte skapa output-video: {video_output}")

print("Startar bearbetning... detta kan ta lite tid beroende på videons längd.")

# ----------------------
# Processa varje frame
# ----------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO-prediction
    results = model.predict(frame, conf=0.5, verbose=False)
    annotated_frame = results[0].plot()  # Rita ut rutor på framen

    # Skriv till output-video
    out.write(annotated_frame)

# ----------------------
# Stäng filer
# ----------------------
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Färdig! Videon sparad som: {video_output}")


In [ ]:
import cv2
from ultralytics import YOLO
import os

video_input = "/Users/gabrieledholm/Documents/PythonML/Pitstop.mp4"
video_output = "/Users/gabrieledholm/Desktop/Pitstop_test.mp4"  # skrivbar plats

cap = cv2.VideoCapture(video_input)
if not cap.isOpened():
    raise RuntimeError("Kunde inte öppna input-video")

fps = cap.get(cv2.CAP_PROP_FPS) or 30
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# H.264 codec
fourcc = cv2.VideoWriter_fourcc(*'avc1')
out = cv2.VideoWriter(video_output, fourcc, fps, (width, height))
if not out.isOpened():
    raise RuntimeError("Kunde inte skapa output-video – kolla codec och skrivbehörighet")

model = YOLO("runs/detect/HP_Logo_train/weights/best.pt")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    results = model.predict(frame, conf=0.3, imgsz=640, verbose=False)
    annotated_frame = results[0].plot()
    out.write(annotated_frame)

cap.release()
out.release()
print("Färdig! Videon sparad som:", video_output)


In [ ]:
import cv2
from ultralytics import YOLO
import os

video_input = "/Users/gabrieledholm/Documents/PythonML/Pitstop.mp4"
video_output = "/Users/gabrieledholm/Desktop/Pitstop_test_with_time1.mp4"

cap = cv2.VideoCapture(video_input)
if not cap.isOpened():
    raise RuntimeError("Kunde inte öppna input-video")

fps = cap.get(cv2.CAP_PROP_FPS) or 30
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'avc1')  # H.264
out = cv2.VideoWriter(video_output, fourcc, fps, (width, height))
if not out.isOpened():
    raise RuntimeError("Kunde inte skapa output-video – kolla codec och skrivbehörighet")

model = YOLO("runs/detect/newHP_Logo_train/weights/best.pt")

# ----------------------
# Timer variabler
# ----------------------
exposure_time = 0.0   # ackumulerad tid i sekunder
frame_time = 1.0 / fps

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(frame, conf=0.3, imgsz=640, verbose=False)

    # Kolla om vi har någon detektion
    if len(results[0].boxes) > 0:
        exposure_time += frame_time  # lägg till tid för denna frame

    # Rita ut YOLO-resultaten
    annotated_frame = results[0].plot()

    # Lägg till timertext på bilden
    timer_text = f"Exposure time: {exposure_time:.2f} s"
    cv2.putText(
        annotated_frame,
        timer_text,
        (50, 50),  # position (x, y)
        cv2.FONT_HERSHEY_SIMPLEX,
        1.2,        # textstorlek
        (0, 0, 255),  # färg (röd)
        3           # tjocklek
    )

    out.write(annotated_frame)

cap.release()
out.release()
print("Färdig! Videon sparad som:", video_output)
